In [ ]:
import numpy as np
import pandas as pd
from sklearn import metrics
import transformers
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW
from transformers import AutoTokenizer, AutoModelForMaskedLM, XLMRobertaTokenizer
from transformers import XLMRobertaTokenizer, XLMRobertaModel


In [ ]:

# Define your configuration variables
MAX_LEN = 256
TRAIN_BATCH_SIZE = 16
VALID_BATCH_SIZE = 4
EPOCHS = 5
LEARNING_RATE = 2e-05
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load your CSV data for both training and validation
train_df = pd.read_csv("data/Training set (Clinical_dataset).csv")[["story", "anger", "fear", "sadness", "calmness", "disgust", "pleasantness", "eagerness", "joy"]]
val_df = pd.read_csv("data/Validation set (Clinical_dataset).csv")[["story", "anger", "fear", "sadness", "calmness", "disgust", "pleasantness", "eagerness", "joy"]]

# Initialize the RoBERTa tokenizer
tokenizer = AutoTokenizer.from_pretrained('dmis-lab/biobert-base-cased-v1.1')

# Create a custom dataset class
class CustomDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.comment_text = dataframe.story
        self.targets = self.data[list(dataframe.columns[1:])]
        self.max_len = max_len

    def __len__(self):
        return len(self.comment_text)

    def __getitem__(self, index):
        comment_text = str(self.comment_text[index])
        comment_text = " ".join(comment_text.split())

        inputs = self.tokenizer.encode_plus(
            comment_text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding="max_length",
            return_token_type_ids=True,
            truncation=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets.iloc[index].values, dtype=torch.float)
        }



In [ ]:
# Create datasets and data loaders for both training and validation
training_set = CustomDataset(train_df, tokenizer, MAX_LEN)
validation_set = CustomDataset(val_df, tokenizer, MAX_LEN)

train_params = {'batch_size': TRAIN_BATCH_SIZE, 'shuffle': True, 'num_workers': 0}
valid_params = {'batch_size': VALID_BATCH_SIZE, 'shuffle': False, 'num_workers': 0}

training_loader = DataLoader(training_set, **train_params)
validation_loader = DataLoader(validation_set, **valid_params)

# Define the ClinicalBERT model
model = AutoModelForSequenceClassification.from_pretrained('dmis-lab/biobert-base-cased-v1.1', num_labels=8)
model.to(device)

# Define the optimizer
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)

# Define the loss function
def loss_fn(outputs, targets):
    # Extract logits from the outputs
    logits = outputs.logits

    return torch.nn.BCEWithLogitsLoss()(logits, targets)

# Training loop
for epoch in range(EPOCHS):
    model.train()
    train_losses = []

    for data in training_loader:
        ids = data['ids'].to(device, dtype=torch.long)
        mask = data['mask'].to(device, dtype=torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)
        targets = data['targets'].to(device, dtype=torch.float)

        optimizer.zero_grad()
        outputs = model(ids, mask, token_type_ids)
        loss = loss_fn(outputs, targets)
        train_losses.append(loss.item())
        loss.backward()
        optimizer.step()

    avg_train_loss = sum(train_losses) / len(train_losses)
    print(f'Epoch {epoch + 1}, Training Loss: {avg_train_loss:.4f}')


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dmis-lab/biobert-base-cased-v1.1 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1, Training Loss: 0.3064
Epoch 2, Training Loss: 0.1905
Epoch 3, Training Loss: 0.1508
Epoch 4, Training Loss: 0.1253
Epoch 5, Training Loss: 0.1017


In [ ]:
from sklearn.metrics import hamming_loss
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
def validate_model():
    model.eval()
    val_losses = []

    val_targets = []
    val_predictions = []

    with torch.no_grad():
        for data in validation_loader:
            ids = data['ids'].to(device, dtype=torch.long)
            mask = data['mask'].to(device, dtype=torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)
            targets = data['targets'].to(device, dtype=torch.float)

            outputs = model(ids, mask, token_type_ids)
            loss = loss_fn(outputs, targets)
            val_losses.append(loss.item())

            val_predictions.extend(outputs.logits.cpu().numpy())
            val_targets.extend(targets.cpu().numpy())

    # Convert val_targets and val_predictions to NumPy arrays
    val_targets = np.array(val_targets)
    val_predictions = np.array(val_predictions)

    # Apply threshold of 0.5 to convert to binary values
    val_predictions = (val_predictions >= 0.5).astype(int)

    # Calculate Hamming Loss
    hamming_loss_value = hamming_loss(val_targets, val_predictions)

    # Calculate Exact Match Ratio
    exact_match_ratio = accuracy_score(val_targets, val_predictions)

    print(f'Hamming Loss: {hamming_loss_value:.4f}')
    print(f'Exact Match Ratio: {exact_match_ratio:.4f}')

    # Calculate and print accuracy, precision, recall, and F1-score for each class
    for i, target_name in enumerate(val_df.columns[1:]):
        target_labels = val_targets[:, i]
        predicted_labels = (val_predictions[:, i] > 0.5)

        accuracy = accuracy_score(target_labels, predicted_labels)
        precision = precision_score(target_labels, predicted_labels, zero_division=0)
        recall = recall_score(target_labels, predicted_labels)
        f1 = f1_score(target_labels, predicted_labels)

        print(f'Class: {target_name}')
        print(f'Accuracy: {accuracy:.4f}')
        print(f'Precision: {precision:.4f}')
        print(f'Recall: {recall:.4f}')
        print(f'F1-Score: {f1:.4f}')

    # Return the average validation loss
    return sum(val_losses) / len(val_losses)

In [ ]:
# Validate the model
validate_model()


Hamming Loss: 0.0652
Exact Match Ratio: 0.7403
Class: anger
Accuracy: 0.9342
Precision: 0.7864
Recall: 0.6983
F1-Score: 0.7397
Class: fear
Accuracy: 0.9224
Precision: 0.8877
Recall: 0.8328
F1-Score: 0.8594
Class: sadness
Accuracy: 0.9166
Precision: 0.9379
Recall: 0.8481
F1-Score: 0.8907
Class: calmness
Accuracy: 0.9397
Precision: 0.9539
Recall: 0.8465
F1-Score: 0.8970
Class: disgust
Accuracy: 0.9440
Precision: 0.9423
Recall: 0.7825
F1-Score: 0.8550
Class: pleasantness
Accuracy: 0.9501
Precision: 0.8636
Recall: 0.6483
F1-Score: 0.7406
Class: eagerness
Accuracy: 0.9241
Precision: 0.9257
Recall: 0.8002
F1-Score: 0.8584
Class: joy
Accuracy: 0.9475
Precision: 0.9469
Recall: 0.9194
F1-Score: 0.9330


0.17776509298434892

In [ ]:
# Save the trained model
model.save_pretrained('BioBERT_trained_model')